# O1v5-SONATA connectome manipulation preparation for enhanced connectome structure

Enhanced connectome structure is obtained by Michael R.'s heuristic rewiring algorithm, producing a set of enhanced adjacency matrices. The algorithm adds and deletes always the same number of connections. Therefore, the resulting number of connections will match exactly the original connectome.

---

This is an auxiliary notebook for configuring rewiring, which...
 - creates connectivity models from adjacency matrices containing additional (structured) connections
 - sets up rewiring using these connectivity models to add reciprocal connections to the original connectome

<u>Requirements</u>: [connectome-manipulator](https://bbpgitlab.epfl.ch/conn/structural/connectome_manipulator) package (v0.0.10.dev1 or later)

ℹ️ Related ticket: [[ACCS-64](https://bbpteam.epfl.ch/project/issues/browse/ACCS-64)] "Rewiring connections to enhance structure in V5 connectome"


In [1]:
# Initialization
import json
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
import tqdm
from bluepysnap import Circuit
from connectome_manipulator.model_building import conn_prob_adj
from scipy import sparse as sps

__Load original circuit & adjacency matrix__

In [2]:
# Load circuit & node ids corresponding to adjacency matrix
circuit_path = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA'
circuit_name = os.path.split(circuit_path)[-1]
circuit_config = os.path.join(circuit_path, 'sonata', 'circuit_config_tmp.json') # SONATA config (.json)  # TEMP edges file with afferent_center_x/y/z properties preliminarily added w/o validation, see [NSETM-1222]

c = Circuit(circuit_config)
nodes = c.nodes["default"]
edges = c.edges["default"]

mc2_nodeset = "mc2_Column"
nids = nodes.ids(mc2_nodeset)
nids_exc = np.intersect1d(nids, nodes.ids({"synapse_class": "EXC"}))

In [3]:
## Original (full) adjacency matrix from baseline sims (for checks)
adj_orig_full = sps.load_npz("/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_Baseline/working_dir/connectivity.npz")
assert adj_orig_full.shape[0] == adj_orig_full.shape[1] == len(nids), "ERROR: Nodes inconsistent with size of original adjacency matrix!"
orig_ccount_full = adj_orig_full.count_nonzero()
print(f'Loaded "original" adjacency matrix with {orig_ccount_full} edges between {adj_orig_full.shape[0]}x{adj_orig_full.shape[1]} nodes')

Loaded "original" adjacency matrix with 7803528 edges between 31346x31346 nodes


__Load and check adjacency matrix with connections to be added__

- Matrices are defined between all MC2 neurons
- Sub-matrix between EXC neurons will be extracted

In [5]:
# Load enhanced (full) adjacency matrices
adj_path = "/gpfs/bbp.cscs.ch/project/proj102/egas/reliability/manipulation_selection/data"
enhance_N = [100, 200, 300, 400, 500, 670]
adj_files = [f"V5_enhanced_mat_{n}k.npz" for n in enhance_N]

adj_mats_full = []
for fn, n in zip(adj_files, enhance_N):
    _adj = sps.load_npz(os.path.join(adj_path, fn))
    assert _adj.shape[0] == _adj.shape[1] == len(nids), "ERROR: Size of enhanced adjacency matrix inconsistent with nodes!"
    adj_mats_full.append(_adj)
    _diff = _adj.count_nonzero() - orig_ccount_full
    print(f'Loaded enhanced "{n}K" adjacency matrix with {_adj.count_nonzero()} edges between {_adj.shape[0]}x{_adj.shape[1]} nodes (DIFF to orig: {_diff}; {100 * _diff / orig_ccount_full:.3f}%)')

Loaded enhanced "100K" adjacency matrix with 7803528 edges between 31346x31346 nodes (DIFF to orig: 0; 0.000%)
Loaded enhanced "200K" adjacency matrix with 7803528 edges between 31346x31346 nodes (DIFF to orig: 0; 0.000%)
Loaded enhanced "300K" adjacency matrix with 7803528 edges between 31346x31346 nodes (DIFF to orig: 0; 0.000%)
Loaded enhanced "400K" adjacency matrix with 7803528 edges between 31346x31346 nodes (DIFF to orig: 0; 0.000%)
Loaded enhanced "500K" adjacency matrix with 7803528 edges between 31346x31346 nodes (DIFF to orig: 0; 0.000%)
Loaded enhanced "670K" adjacency matrix with 7803528 edges between 31346x31346 nodes (DIFF to orig: 0; 0.000%)


In [9]:
# Extract EXC-EXC sub-matrix
EE_sel = np.isin(nids, nids_exc)
adj_orig = adj_orig_full[:, EE_sel][EE_sel, :]
assert adj_orig.shape[0] == adj_orig.shape[1] == len(nids_exc), "ERROR: Size of original adjacency matrix inconsistent with EXC nodes!"
orig_ccount = adj_orig.count_nonzero()
print(f'Extracted "original" adjacency matrix with {orig_ccount} edges between {adj_orig.shape[0]}x{adj_orig.shape[1]} nodes')

adj_mats = []
for _adj, _n in zip(adj_mats_full, enhance_N):
    _adj = _adj[:, EE_sel][EE_sel, :]
    assert _adj.shape[0] == _adj.shape[1] == len(nids_exc), "ERROR: Size of enhanced adjacency matrix inconsistent with EXC nodes!"
    adj_mats.append(_adj)
    _diff = _adj.count_nonzero() - orig_ccount
    print(f'Extracted enhanced "{_n}K" adjacency matrix with {_adj.count_nonzero()} edges between {_adj.shape[0]}x{_adj.shape[1]} EXC nodes (DIFF to orig: {_diff}; {100 * _diff / orig_ccount:.3f}%)')

Extracted "original" adjacency matrix with 6717001 edges between 26567x26567 nodes
Extracted enhanced "100K" adjacency matrix with 6717001 edges between 26567x26567 EXC nodes (DIFF to orig: 0; 0.000%)
Extracted enhanced "200K" adjacency matrix with 6717001 edges between 26567x26567 EXC nodes (DIFF to orig: 0; 0.000%)
Extracted enhanced "300K" adjacency matrix with 6717001 edges between 26567x26567 EXC nodes (DIFF to orig: 0; 0.000%)
Extracted enhanced "400K" adjacency matrix with 6717001 edges between 26567x26567 EXC nodes (DIFF to orig: 0; 0.000%)
Extracted enhanced "500K" adjacency matrix with 6717001 edges between 26567x26567 EXC nodes (DIFF to orig: 0; 0.000%)
Extracted enhanced "670K" adjacency matrix with 6717001 edges between 26567x26567 EXC nodes (DIFF to orig: 0; 0.000%)


In [10]:
# Check edges to be added/deleted
for _adj, _n in zip(adj_mats, enhance_N):
    _diff_mat = _adj.astype(int) - adj_orig.astype(int)
    assert not np.any(_diff_mat.data == 0), 'ERROR: Zero check!'
    print(f'Enhanced "{_n}K" adjacency matrix: {_diff_mat.count_nonzero()} edges changed ({np.sum(_diff_mat.data == 1)} added, {np.sum(_diff_mat.data == -1)} deleted)')


Enhanced "100K" adjacency matrix: 191326 edges changed (95663 added, 95663 deleted)
Enhanced "200K" adjacency matrix: 381620 edges changed (190810 added, 190810 deleted)
Enhanced "300K" adjacency matrix: 570518 edges changed (285259 added, 285259 deleted)
Enhanced "400K" adjacency matrix: 758446 edges changed (379223 added, 379223 deleted)
Enhanced "500K" adjacency matrix: 945906 edges changed (472953 added, 472953 deleted)
Enhanced "670K" adjacency matrix: 1261302 edges changed (630651 added, 630651 deleted)


__Create and save adjacency model for rewiring__

In [11]:
adj_model_path = f"/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/enhanced_connectivity/model_building/{circuit_name}/model"
if not os.path.exists(adj_model_path):
    os.makedirs(adj_model_path)

In [12]:
adj_model_files = []
for _adj, _n in zip(adj_mats, enhance_N):
    adj_model = conn_prob_adj.build(_adj.tocsc(), nids_exc, nids_exc)
    adj_model_name = f"ConnProbAdjModel_mc2EE_Enhanced{_n}K"
    adj_model.save_model(adj_model_path, adj_model_name)
    print(adj_model)
    adj_model_file = os.path.join(adj_model_path, adj_model_name + ".json")
    adj_model_files.append(adj_model_file)
    assert os.path.exists(adj_model_file), f'ERROR: Model file "{adj_model_file}" not saved!'
    print(f'Model saved to "{adj_model_file}"\n')

ConnProbAdjModel
  <26567x26567 sparse matrix of type '<class 'numpy.bool_'>'
	with 6717001 stored elements in Compressed Sparse Column format>
Model saved to "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/enhanced_connectivity/model_building/O1v5-SONATA/model/ConnProbAdjModel_mc2EE_Enhanced100K.json"

ConnProbAdjModel
  <26567x26567 sparse matrix of type '<class 'numpy.bool_'>'
	with 6717001 stored elements in Compressed Sparse Column format>
Model saved to "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/enhanced_connectivity/model_building/O1v5-SONATA/model/ConnProbAdjModel_mc2EE_Enhanced200K.json"

ConnProbAdjModel
  <26567x26567 sparse matrix of type '<class 'numpy.bool_'>'
	with 6717001 stored elements in Compressed Sparse Column format>
Model saved to "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/enhanced_connectivity/model_building/O1v5-SONATA/model/ConnProbAdjModel_mc2EE_Enhanced300K.json"

ConnProbAdjModel
  <26567x26567 sparse matrix of type '<class 'numpy.b

__Create rewiring config & launch rewiring__

In [13]:
output_base_path = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits'
config_paths = ['../configs', '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/enhanced_connectivity/configs']  # Write to local and proj9 folder

for _cpath in config_paths:
    if not os.path.exists(_cpath):
        os.makedirs(_cpath)

In [14]:
def default_manip_config(circuit_config, seed=3210, N_split=None):
    """Generates a default manipulation config dict w/o any specific manipulation."""
    manip_config = {}
    manip_config['circuit_config'] = circuit_config
    manip_config['seed'] = seed
    if N_split is not None:
        manip_config['N_split_nodes'] = N_split
    return manip_config

def add_manip_to_config(manip_config, name, prob_model_file, delay_model_file, props_model_file):
    """Adds an EE rewiring operation based to the manipulation config dict (in-place)."""
    assert 'manip' not in manip_config, "ERROR: Manipulation operation already specified!"
    manip_config['manip'] = {'name': f'ConnRewire{name}mc2EE',
                             'fcts': [{'source': 'conn_rewiring',
                                       'sel_src': {'hypercolumn': 2, 'synapse_class': 'EXC'},
                                       'sel_dest': {'hypercolumn': 2, 'synapse_class': 'EXC'},
                                       'syn_class': 'EXC',
                                       'keep_indegree': False,
                                       'reuse_conns': False,
                                       'keep_conns': True,
                                       'reuse_pos': True,
                                       'rewire_mode': None,
                                       'gen_method': 'randomize',
                                       'amount_pct': 100.0,
                                       'model_config': {'prob_model_spec': {'file': prob_model_file},
                                                        'delay_model_spec': {'file': delay_model_file},
                                                        'props_model_spec': {'file': props_model_file}}}]}

def export_manip_config(manip_config, config_path, print_cmd=False, circuit_name=None, output_base_path=None, N_parallel=None):
    """Writes manipulation config to .json config file(s)."""
    if not isinstance(config_path, list):
        config_path = [config_path]

    fn = f'manip_config__{manip_config["manip"]["name"]}.json'
    for cpath in config_path:
        with open(os.path.join(cpath, fn), 'w') as f:
            json.dump(manip_config, f, indent=2)
        print(f"Config file {fn} written to {cpath}")

    if print_cmd:
        assert circuit_name is not None and output_base_path is not None and N_parallel is not None, \
            "ERROR: circuit_name/output_base_path/N_parallel required for printing launch command!"
        print()
        output_dir = os.path.join(output_base_path, circuit_name + f'__{manip_config["manip"]["name"]}')
        print_launch_cmd(cpath, fn, output_dir, N_parallel)

def print_launch_cmd(config_path, config_fn, output_dir, N_parallel):
    run_cmd = f'sbatch run_rewiring_parallel.sh "{config_fn}" "{output_dir}" {N_parallel}'
    print(f"# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]")
    print(f"cd {config_path}")
    print(run_cmd)


In [15]:
# Model locations (MC2 column models!!)
delay_model_file = os.path.join('/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/model_building', circuit_name, 'model', 'DistDepDelayO1v5mc2EE.json')
props_model_file = os.path.join('/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/model_building', circuit_name, 'model', 'ConnPropsPerPathwayO1v5mc2EE.json')  # Latest model format
print('prob_model_file =')
adj_model_files

prob_model_file =


['/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/enhanced_connectivity/model_building/O1v5-SONATA/model/ConnProbAdjModel_mc2EE_Enhanced100K.json',
 '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/enhanced_connectivity/model_building/O1v5-SONATA/model/ConnProbAdjModel_mc2EE_Enhanced200K.json',
 '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/enhanced_connectivity/model_building/O1v5-SONATA/model/ConnProbAdjModel_mc2EE_Enhanced300K.json',
 '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/enhanced_connectivity/model_building/O1v5-SONATA/model/ConnProbAdjModel_mc2EE_Enhanced400K.json',
 '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/enhanced_connectivity/model_building/O1v5-SONATA/model/ConnProbAdjModel_mc2EE_Enhanced500K.json',
 '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/enhanced_connectivity/model_building/O1v5-SONATA/model/ConnProbAdjModel_mc2EE_Enhanced670K.json']

In [16]:
for _prob_file, _n in zip(adj_model_files, enhance_N):
    manip_config = default_manip_config(circuit_config, seed=3210)
    add_manip_to_config(manip_config, f'Enhanced{_n}K', _prob_file, delay_model_file, props_model_file)
    export_manip_config(manip_config, config_paths, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500)
    if _n != enhance_N[-1]:
        print('----------------------------------------------------------------------------------------------------')

Config file manip_config__ConnRewireEnhanced100Kmc2EE.json written to ../configs
Config file manip_config__ConnRewireEnhanced100Kmc2EE.json written to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/enhanced_connectivity/configs

# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]
cd /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/enhanced_connectivity/configs
sbatch run_rewiring_parallel.sh "manip_config__ConnRewireEnhanced100Kmc2EE.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireEnhanced100Kmc2EE" 500
----------------------------------------------------------------------------------------------------
Config file manip_config__ConnRewireEnhanced200Kmc2EE.json written to ../configs
Config file manip_config__ConnRewireEnhanced200Kmc2EE.json written to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/enhanced_connectivity/configs

# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]
cd /gpfs/bbp.cscs.ch/pr

Launch commands:

`cd /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/enhanced_connectivity/configs`

`sbatch run_rewiring_parallel.sh "manip_config__ConnRewireEnhanced100Kmc2EE.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireEnhanced100Kmc2EE" 500`

`sbatch run_rewiring_parallel.sh "manip_config__ConnRewireEnhanced200Kmc2EE.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireEnhanced200Kmc2EE" 500`

`sbatch run_rewiring_parallel.sh "manip_config__ConnRewireEnhanced300Kmc2EE.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireEnhanced300Kmc2EE" 500`

`sbatch run_rewiring_parallel.sh "manip_config__ConnRewireEnhanced400Kmc2EE.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireEnhanced400Kmc2EE" 500`

`sbatch run_rewiring_parallel.sh "manip_config__ConnRewireEnhanced500Kmc2EE.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireEnhanced500Kmc2EE" 500`

`sbatch run_rewiring_parallel.sh "manip_config__ConnRewireEnhanced670Kmc2EE.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireEnhanced670Kmc2EE" 500`


### AFTER REWIRING

__Extract actual connection counts (from log files)__

In [22]:
# Check connection/synapse counts
from datetime import datetime, timedelta

N_split = 500
# main_log_file = "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireEnhanced100Kmc2EE/logs/connectome_manipulation_2023-12-06_08h27.log"
# main_log_file = "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireEnhanced200Kmc2EE/logs/connectome_manipulation_2023-12-06_08h27.log"
# main_log_file = "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireEnhanced300Kmc2EE/logs/connectome_manipulation_2023-12-06_08h27.log"
# main_log_file = "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireEnhanced400Kmc2EE/logs/connectome_manipulation_2023-12-06_08h27.log"
# main_log_file = "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireEnhanced500Kmc2EE/logs/connectome_manipulation_2023-12-06_08h27.log"
main_log_file = "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireEnhanced670Kmc2EE/logs/connectome_manipulation_2023-12-06_08h28.log"

log_path, log_fn = os.path.split(main_log_file)  # Main log path/filename
log_date = os.path.splitext(log_fn)[0].split('connectome_manipulation_')[1]
log_date_alt = log_date.split('_')
log_date_alt[-1] = (datetime.strptime(log_date_alt[-1], '%Hh%M') + timedelta(minutes=1)).strftime('%Hh%M')
log_date_alt = '_'.join(log_date_alt)  # Alternative time (since log file names sometimes off by 1 minute)

log_folders = [os.path.join(log_path, d) for d in os.listdir(log_path) if os.path.isdir(os.path.join(log_path, d))]
data_log_splits = [f'{log_date}.RewiringIndices_{i + 1}_{N_split}.npz' for i in range(N_split)]
data_log_splits_alt = [f'{log_date_alt}.RewiringIndices_{i + 1}_{N_split}.npz' for i in range(N_split)]  # Alternative names

conn_count_orig_all = 0 # Overall input connection count
conn_count_actu_all = 0 # Overall output connection count after rewiring (actual value)
conn_count_orig_sel = 0 # Input connection count within selected rewiring target
conn_count_actu_sel = 0 # Output connection count within selected rewiring target after rewiring (actual value)
split_count = 0
for split_name, alt_name in tqdm.tqdm(zip(data_log_splits, data_log_splits_alt), desc="Collecting log files", total=N_split):
    # Search for log file in all log folders
    folder_name = None
    file_name = None
    for sp_nm in [split_name, alt_name]:
        if sp_nm is None:
            break
        for fld in log_folders:
            tmp_name = [fn for fn in os.listdir(fld) if sp_nm in fn]
            assert len(tmp_name) <= 1, 'ERROR: Multiple files found in current folder!'
            if len(tmp_name) == 1:  # Match
                assert folder_name is None, 'ERROR: Folder not unique!'
                assert file_name is None, 'ERROR: File not unique!'
                folder_name = fld
                file_name = tmp_name[0]
        if folder_name is not None and file_name is not None:
            break
    assert folder_name is not None and file_name is not None, f'ERROR: Log file ..{split_name} not found!'

    # Read log file
    file_name = file_name.replace("Indices", "Stats")  # In any case, the data log to load is the ..Stats.. one, if existing
    dlog = os.path.join(folder_name, file_name)
    if not os.path.exists(dlog):
        continue
    split_count += 1
    stats_dict = np.load(dlog)
    conn_count_orig_sel += np.sum(stats_dict['input_conn_count_sel'])
    conn_count_orig_all += np.sum(stats_dict.get('input_conn_count', 0))
    conn_count_actu_sel += np.sum(stats_dict['output_conn_count_sel'])
    assert stats_dict['output_conn_count'] == np.sum(stats_dict['output_conn_count_sel']) + np.sum(stats_dict['input_conn_count']) - np.sum(stats_dict['input_conn_count_sel'])
    conn_count_actu_all += stats_dict['output_conn_count']

print(f'Non-empty splits: {split_count}/{N_split}')
print(f'Overall: #Conns_orig = {conn_count_orig_all}, #Conns_rewired = {conn_count_actu_all}, DIFF = {conn_count_actu_all - conn_count_orig_all} ({100.0 * (conn_count_actu_all - conn_count_orig_all) / conn_count_orig_all:.2f}%)')
print(f'Within wiring target: #Conns_orig = {conn_count_orig_sel}, #Conns_rewired = {conn_count_actu_sel}, DIFF = {conn_count_actu_sel - conn_count_orig_sel} ({100.0 * (conn_count_actu_sel - conn_count_orig_sel) / conn_count_orig_sel:.2f}%)')

Non-empty splits: 72/500
Overall: #Conns_orig = 14138043, #Conns_rewired = 14138043, DIFF = 0 (0.00%)
Within wiring target: #Conns_orig = 6717001, #Conns_rewired = 6717001, DIFF = 0 (0.00%)


Rewiring statistics:

- `ConnRewireEnhanced100Kmc2EE`: #Conns = 6717001 (DIFF = 0), #Syns = 356901714 synapses (DIFF: -13595)
- `ConnRewireEnhanced200Kmc2EE`: #Conns = 6717001 (DIFF = 0), #Syns = 356888639 synapses (DIFF: -26670)
- `ConnRewireEnhanced300Kmc2EE`: #Conns = 6717001 (DIFF = 0), #Syns = 356876553 synapses (DIFF: -38756)
- `ConnRewireEnhanced400Kmc2EE`: #Conns = 6717001 (DIFF = 0), #Syns = 356865138 synapses (DIFF: -50171)
- `ConnRewireEnhanced500Kmc2EE`: #Conns = 6717001 (DIFF = 0), #Syns = 356852346 synapses (DIFF: -62963)
- `ConnRewireEnhanced670Kmc2EE`: #Conns = 6717001 (DIFF = 0), #Syns = 356824988 synapses (DIFF: -90321)

### AFTER PREPROCESSING

__Check connectomes__

(Full MC2 connectomes)

In [2]:
# # Load circuit & node ids corresponding to adjacency matrix
# c = Circuit("/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA/sonata/circuit_config_tmp.json")
# nodes = c.nodes["default"]
# mc2_nodeset = "mc2_Column"
# nids = nodes.ids(mc2_nodeset)

In [ ]:
# # Adjacency list
# enhance_N = [100, 200, 300, 400, 500]
# adj_list = [f"/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/ConnRewireEnhanced{_n}Kmc2EE/working_dir/connectivity.npz" for _n in enhance_N]
# ref_list = [f"/gpfs/bbp.cscs.ch/project/proj102/egas/reliability/manipulation_selection/data/V5_enhanced_mat_{_n}k.npz" for _n in enhance_N]

# for adj_file, ref_file in zip(adj_list, ref_list):
#     print(f"CHECKING {adj_file}...", end="")
#     adj = sps.load_npz(adj_file)
#     ref_adj = sps.load_npz(ref_file)

#     assert np.array_equal(adj.toarray(), ref_adj.toarray()), "ERROR: Adjacency mismatch!"
#     print("OK")